In [1]:
#!pip install torchsummary 

In [2]:
# Revisar este video para las consideraciones de si guardas en CPU o GPU
#https://www.youtube.com/watch?v=9L9jEOwRrCg

In [3]:


import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchsummary import summary

In [4]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
##num_epochs = 25
num_epochs = 3
batch_size = 100
learning_rate = 0.001

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Files already downloaded and verified


In [5]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [6]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(conv3x3(self.in_channels, out_channels, stride=stride),
                                       nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [7]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
    
print()

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])


Model's state_dict:
conv.weight 	 torch.Size([16, 3, 3, 3])
bn.weight 	 torch.Size([16])
bn.bias 	 torch.Size([16])
bn.running_mean 	 torch.Size([16])
bn.running_var 	 torch.Size([16])
bn.num_batches_tracked 	 torch.Size([])
layer1.0.conv1.weight 	 torch.Size([16, 16, 3, 3])
layer1.0.bn1.weight 	 torch.Size([16])
layer1.0.bn1.bias 	 torch.Size([16])
layer1.0.bn1.running_mean 	 torch.Size([16])
layer1.0.bn1.running_var 	 torch.Size([16])
layer1.0.bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv2.weight 	 torch.Size([16, 16, 3, 3])
layer1.0.bn2.weight 	 torch.Size([16])
layer1.0.bn2.bias 	 torch.Size([16])
layer1.0.bn2.running_mean 	 torch.Size([16])
layer1.0.bn2.running_var 	 torch.Size([16])
layer1.0.bn2.num_batches_tracked 	 torch.Size([])
layer1.1.conv1.weight 	 torch.Size([16, 16, 3, 3])
layer1.1.bn1.weight 	 torch.Size([16])
layer1.1.bn1.bias 	 torch.Size([16])
layer1.1.bn1.running_mean 	 torch.Size([16])
layer1.1.bn1.running_var 	 torch.Size([16])
layer1.1.bn1.num_batches_tr

In [9]:
#Detalle de los tensores de los parametros
#for param in model.parameters():
#    print(param)

In [10]:
decay = 0
model.train()
for epoch in range(num_epochs):
    
    # Decay the learning rate every 20 epochs
    if (epoch+1) % 20 == 0:
        decay+=1
        optimizer.param_groups[0]['lr'] = learning_rate * (0.5**decay)
        print("The new learning rate is {}".format(optimizer.param_groups[0]['lr']))
        
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
            # para cada epoca guarda el avance
            EPOCH = epoch+1
            # Guardar checkpoint
            PATH = "RestNet_TP_epoca_[{}]".format(EPOCH)
            LOSS = loss.item()

            torch.save({
               'epoch': EPOCH,
               'model_state_dict': model.state_dict(),
               'optimizer_state_dict': optimizer.state_dict(),
               'loss': LOSS,
                }, PATH)


Epoch [1/3], Step [100/500] Loss: 1.4292
Epoch [1/3], Step [200/500] Loss: 1.4359
Epoch [1/3], Step [300/500] Loss: 1.3116
Epoch [1/3], Step [400/500] Loss: 1.1250
Epoch [1/3], Step [500/500] Loss: 1.0609
Epoch [2/3], Step [100/500] Loss: 0.9956
Epoch [2/3], Step [200/500] Loss: 1.0029
Epoch [2/3], Step [300/500] Loss: 0.9559
Epoch [2/3], Step [400/500] Loss: 0.7956
Epoch [2/3], Step [500/500] Loss: 1.0043
Epoch [3/3], Step [100/500] Loss: 1.1316
Epoch [3/3], Step [200/500] Loss: 0.7942
Epoch [3/3], Step [300/500] Loss: 0.8083
Epoch [3/3], Step [400/500] Loss: 0.8620
Epoch [3/3], Step [500/500] Loss: 0.6648


# Guarda el modelo entrenado

In [11]:
# Training End TE
torch.save(model, './RestNet_TE.pth')

In [12]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 69.67 %


In [13]:
summary(model, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 16, 32, 32]           2,304
       BatchNorm2d-5           [-1, 16, 32, 32]              32
              ReLU-6           [-1, 16, 32, 32]               0
            Conv2d-7           [-1, 16, 32, 32]           2,304
       BatchNorm2d-8           [-1, 16, 32, 32]              32
              ReLU-9           [-1, 16, 32, 32]               0
    ResidualBlock-10           [-1, 16, 32, 32]               0
           Conv2d-11           [-1, 16, 32, 32]           2,304
      BatchNorm2d-12           [-1, 16, 32, 32]              32
             ReLU-13           [-1, 16, 32, 32]               0
           Conv2d-14           [-1, 16,

In [ ]:
break; 

# Timing Pytorch Code Fragments

## Wrong Way

In [ ]:
# Below timing method will NOT work for asynchronous cuda calls
import time as timerstart = timer.time()
loss.backward()
print("Time taken", timer.time() - start)  # tiny value

## Right Way

In [ ]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()# whatever you are timing goes hereend.record()

# Waits for everything to finish running
torch.cuda.synchronize()

print(start.elapsed_time(end))  # milliseconds

## More detail example: https://gist.github.com/aurotripathy/2ee988e9bbaecc78a509fd6a21d3076d

# Saving Combinations between CPU & GPU

## Save GPU and load on CPU

In [ ]:
#Save 
device = torch.device("cuda")
model.to(device)
torch.save(model.state_dict(), PATH)

In [ ]:
# Load
device = torch.device('cpu')
model. Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location=device))

## Save GPU load GPU

In [ ]:
#Save 
device = torch.device("cuda")
model.to(device)
torch.save(model.state_dict(), PATH)

In [ ]:
# Load
model. Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(devide)


## Save CPU load GPU

In [ ]:
# Save
torch.save(model.state_dict(), PATH)

In [ ]:
# Load
device = torch.device("cuda")
model. Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location="cuda"))
model.to(device)